<a href="https://colab.research.google.com/github/bca2/593/blob/master/r_vs_excel_stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Purpose

This document is intended to inform instructors on when and why a statistics package like R should be used over Excel.

Code has been hidden.

There is a .csv file required ("**test2.csv**"), and it should be on github.


# Statistics: R vs Excel

Excel (or some sort of spreadsheet) skills are defininitely something that all students attending a post-secondary institution will need.

Originally, I learned excel for stats, and I still use excel for storing data (many people/organizations do).

However, Excel is not a serious (or even competent) statistics program, and should be avoided if the analysis is anything more complicated than simple linear regression or a t-test (balanced ANOVAs are probably fine too).
Excel was not built to do statistics properly.
Be warned that the statistics Excel claims to do outstrip what it can actually do correctly.

# Non-linear data

Excel does just fine with linear regression but incorrectly fits non-linear data (except maybe polynomial fits, those should just be extensions of a linear fit).

Many non-linear problems in science are non-linear: exponential growth or decay, logistic growth, Michaelis–Menten kinetics, sinusoidal etc.

Here's the thing: Excel doesn't actually fit non-linear models even though it claims to fit exponential, logarithmic, and power models (see the trendline options for graphs) .
Instead, it linearizes the data (transforms the data) so that it's linear, then backtransforms the data.

This is an example of case where it doesn't work.

## Exponential decay

The data we'll be fitting a line to show exponential decay.
A general form of the equation is:

$$y = A\cdot e^{x \cdot  r_{max}}$$

Excel will log-transform the data, fit a linear model, and then backtransform the data.
The model excel will actually fit is:

\begin{align}
\begin{aligned}
\ln(y) &= \ln(A \cdot e^{x \cdot  r_{max}})\\
           &= \ln(A) + x \cdot  r_{max}
\end{aligned}
\end{align}

This transformation is completely legitimate.

### Why linearization usually doesn't work

Unfortunately, these equations are incomplete.
The true equation that represents our data is:

$$y_i = A\cdot e^{x_i \cdot  r_{max}} + \epsilon_i$$

where $y_i$ is the $i^{th}$ data value and $\epsilon_i$ is the error associated with the $i^{th}$ data value.
Errors are assumed to be (and are generally pretty close) normally distributed random values with a mean of 0 and constant variance (to be estimated from the residuals).

$$\epsilon \sim  N(0, \sigma^2)$$

Many statistical procedures assume that the errors follow this unknown distribution.
Simple linear regression, for example, does.
Non-linear regression does, too.

When Excel performs the transformation to linearize the data, the error is transformed.

$$ln(y) = \ln(A\cdot e^{x \cdot  r_{max}}+\epsilon)$$

The transformation is no longer nice and clean.

If the errors were approximately $\epsilon \sim  N(0, \sigma^2)$ before the transformation, then they likely will not be after the transformation.
This means that the simple linear regression that Excel performs on the transformed data is innapropriate since we fail to meet one of the model assumptions (either the residuals are not normal, the variance isn't constant, or both).

Below is an analysis of simulated data output showing the differences between Excel and R.
The simulated data is exponential with $A=100$ and $ r_{max}=-0.025$ with $\epsilon \sim N(0, 10^2)$.


### When linearization would work

If the errors become normally distributed and homogenous due to the transformation then linearization could work.

In [0]:
#@title Activate R
# The following code lets us speak in the R language while working from within Python

%load_ext rpy2.ipython

# Using R inside python in this way tends to produce warnings
# The warnings don't seem to have an effect on the desired output, so I'm just going to suppress the warnings.
# You can go ahead and run the code with and without warnings by excluding/including the following code:

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore", category=RRuntimeWarning)

# To comment out code, just use the pound sign (hashtag) "#"
# Code that's been commented out will not be run, just like this comment I'm writing right now.
# Comments help us in two major ways:
#   1. You can easily leave instructions or clarifications in your code for other people or for future you.
#   2. You can see how your code runs without certain lines of code without actually deleting your code.
#       My advice: Try not to delete your code. You may change your mind later, and it's good to have it readily available.

In [0]:
#@title Import data (test2.csv)
from google.colab import files
uploaded = files.upload()

The "test2.csv" data should be available in the github folder.

In [0]:
#@title Non-linear model fit
%%R

library(nlme)
library(tidyr)
library(dplyr)

exp = read.csv("test2.csv")

exp = exp%>%
    mutate(y_act = 100*exp(-x*.025))

A_start=100

k_start=-.03

fit_exp <- nls(data=exp,
            formula = y ~ A*exp(x*k),
            start = list(A=A_start,k=k_start))
summary(fit_exp)

Note that our estimates are:

\begin{align}
\begin{aligned}
 A&=97.7\\
  r_{max}&=-0.0248\\
 \sigma &= 9.697
\end{aligned}
\end{align}

...which is pretty good.

### Checking the residuals

We generally graph a scatterplot of the residuals to make sure that they're approximately normal and have equal variance.


In [0]:
#@title Residuals from the R fit (residuals seem homogeneous, good)
%%R
plot(fit_exp)

In [0]:
#@title Residuals from the R fit (residuals seem normalish, good)
%%R
qqnorm(residuals(fit_exp))
qqline(residuals(fit_exp))

#### R residual analysis

The residuals seem homogeneous.
The graph looks more or less like a shot-gun blast, it's random and there's no real pattern.

The residuals are normalish (Q-Q plot).
Perfectly normal residuals would all fall on the line, our data is pretty close.

In [0]:
#@title Model fits (R = green, Excel = Red, True equation = blue)
%%R
plot(exp$x,exp$y)
lines(predict(fit_exp), col="green")
lines(exp$x,exp$excel_y, col="red")
lines(exp$x,exp$y_act, col="blue")


The non-linear model fit with R is clearly closer to the true equation.
In this case we know what the true equation is because the data were simulated by me using R.

## Residual analysis of the excel fit

In [0]:
#@title Residuals from the Excel fit (obvious pattern, not good)
%%R

library(dplyr)
library(tidyr)

exp2 = exp%>%
    mutate(excel_res = (excel_y-y)/11.72071)
  
plot(x=exp2$excel_y, y=exp2$excel_res)


In [0]:
#@title Residuals from the Excel fit (residuals seem normalish, maybe not as good as before, but not awful)
%%R
qqnorm(exp2$excel_res)
qqline(exp2$excel_res)

# The Tradeoff

R does the correct analysis, however, a non-linear analyis doesn't necessarily just work out of the box.

Non-linear models require starting values, which can be a pain.
Linear models are so straight forward that you could do them by hand with pen and paper (if you have some linear algebra in your background).

Fitting non-linear models is sort of a guessing game.
The algorithm fitting the model will take the starting values that you provide and attempt to improve upon them by making tiny changes.
Improvement here means reducing the resisdual sums of squares to the point where it identifies the parameter values that result in the minimum sums of squares (identical to the goal of a simple linear model).

This can be a little finicky because there may be multiple local minima even though there's only one absolute minimum.
If the starting parameters are incorrectly specified then the algorithm can get stuck on a local minimum resulting in poor estimates.
Additionally there can be other issues, especially with complex models.
This isn't a reason not to do a non-linear models, but it's a reason to keep the non-linear models simple for high school.

## Guessing good starting values: could be a good exercise for students

We need to guess good starting values for the non-linear model to work properly.

Let's start with $A$.

$$y = A\cdot e^{x \cdot k}$$

If $x=0$ then $y=A$ (because $e^0=1$).

We can probably get a good guess of $A$ from a scatterplot.

In [0]:
#@title Guessing A from a graph
%%R
library(ggplot2)

plot = ggplot(exp, aes(x = x, y = y))
plot = plot + geom_point()
plot

A good starting value of $A$ would be around 100.

#### $r_{max}$ is a bit tougher, we can't just easily guess it from the graph

We can cheat a little bit and use linearization to get an estimate of $k$.
This is good for getting in the right ballpark, but it's not our final estimate.

Use this transformation on all observations and then take the average.

$$\frac{ln(y_i)-\ln(A)}{x_i}= r_{max}$$
           
...which is about $-0.03$.


###  Linearized model to get starting values

We can also just fit the linear model (this is what excel does) and use those estimates in our non-linear model.
This won't work for every non-linear model, but it will with this exponential model.

The log-transformed model (approximately) looks like this:

$$\ln{y}=\ln{A} + r_{max} \cdot x$$



Notice that if we fit this linear model, our intercept $\beta_0 = \ln{A}$ and our slope $\beta_1 = r_{max}$.
For this model these are probably pretty good starting values.

$r_{max}$ seems pretty easy to find.To find $A$:

\begin{align}
\begin{aligned}
\beta_0 &= \ln{A}\\
    e^{\beta_0}&= A
\end{aligned}
\end{align}